In [43]:
import cython
import sympy
import numpy as np 
import matplotlib.pyplot as plt

%load_ext cython

FS=(12,12)

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [7]:
def is_prime(n):
    if n <= 1:
        return 0
    if n <= 3:
        return 1
    else:
        if np.mod(n,2) == 0 or np.mod(n,3 ) == 0:
            return 0

        upper_bound = int(np.floor(np.sqrt(n)))

        d = 5

        while d <= upper_bound:
            if np.mod(n,d) == 0 or np.mod(n,d + 2) == 0:
                return 0
            d += 6

    return 1

In [39]:
%%cython --compile-args=-Ofast
import numpy as np
cimport numpy as np 

# Add the prime testing function
cdef np.uint8_t is_prime_cython(size_t n):
    cdef size_t i, upper_bound
    
    if n <= 1:
        return 0
    if n <= 3:
        return 1
    if n % 2 == 0 or n % 3 == 0:
        return 0
    
    upper_bound = <size_t>(np.sqrt(n) + 1)
    
    # Check divisibility by numbers of form 6k±1
    i = 5
    while i <= upper_bound:
        if n % i == 0 or n % (i + 2) == 0:
            return 0
        i += 6
        
    return 1

# Add Python wrapper function
def is_prime_cy(n):
    return bool(is_prime_cython(n))

cpdef generate_ulam_spiral(int size):
    cdef:
        size_t num = 1
        size_t step_size = 1
        int x = size // 2, y = size // 2
    
    if np.mod(size, 2) == 0:
        raise ValueError("Size can not be even")
    
    # Fix: zeros, not zeroes
    grid = np.zeros((size,size), dtype=np.uint8)
    directions = [(0,1),(1,0),(0,-1),(-1,0)]
    direction = 0

    while num <= <size_t>(size ** 2):
        for _ in range(2):
            for _ in range(step_size):
                if 0 <= x < size and 0 <= y < size:
                    if is_prime_cy(num):
                        grid[size - y - 1, x] = 1
                
                x += directions[direction][0]
                y += directions[direction][1]
                num += 1

                if num > <size_t>(size ** 2):
                    break
            
            direction = np.mod(direction + 1, 4)
        step_size += 1
    return grid


for i in range(100):
    if is_prime_cy(i):
        print(i)

Content of stderr:
In file included from /Users/ameyassh/.cache/ipython/cython/_cython_magic_0c4fbfe5b4aad1410410bbc11a9b9e6d0f2b977f.c:1269:
In file included from /Users/ameyassh/miniconda3/envs/{EP}/lib/python3.13/site-packages/numpy/_core/include/numpy/arrayobject.h:5:
In file included from /Users/ameyassh/miniconda3/envs/{EP}/lib/python3.13/site-packages/numpy/_core/include/numpy/ndarrayobject.h:12:
In file included from /Users/ameyassh/miniconda3/envs/{EP}/lib/python3.13/site-packages/numpy/_core/include/numpy/ndarraytypes.h:1913:
/Users/ameyassh/miniconda3/envs/{EP}/lib/python3.13/site-packages/numpy/_core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: "Using deprecated NumPy API, disable it with "          "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-W#warnings]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^
/Users/ameyassh/.cache/ipython/cython/_cython_magic_0c4fbfe5b4aad1410410bbc11a9b9e6d0f2b977f.c:7165:21: warning: fallthrough an

In [38]:
for n in range(100):
    if is_prime(n):
        print(n, end=" ")

2 3 5 7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89 97 

In [35]:
def compute_grid_size(n=100):

    size = int(np.ceil(np.sqrt(n)))
    if np.mod(size,2) == 0:
        size += 1
    
    return size

In [ ]:
def plot_ulam_spiral(size=None, n=None, figsize=(8,8)):

    if size is None and n is not None:
        size = compute_grid_size(n)
    elif size is None and n is None:
        raise ValueError("Both size and n are None")
    elif size is not None and n is not None:
        raise ValueError("Both size and n are given")

    spiral = generate_ulam_spiral(size)
    plt.figure(figsize=figsize)
    plt.imshow(spiral, cmap="YlOrBr") #magma cool af
    plt.axis("off")

In [65]:
plot_ulam_spiral(n=10000, figsize=(100,100))